# TF-IDF

There are many ways to vectorize texts. One of them is tf-idf.

## TF-IDF

[tf–idf, TF*IDF, TFIDF, term frequency–inverse document frequency,](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.

### TF

TF -- **text frequency** — the number of times a term (word) occurs in a document. To compute, divide the number of times a term occurs in the document by the number of terms in the document.

$$TF_{i,j} = \frac{n_{i,j}}{\sum_{k}^{}n_{i,j}}$$

### IDF

IDF — **inverse document frequency** —  diminishes the weight of terms that occur very frequently in the document set and increases the weight of terms that occur rarely.

$$ IDF = log \frac{N}{n_t}$$

— N -- the number of documents, n -- the number of documents where the term occurs at least once. If the term occurs only in one document, its IDF will be high (=> the term is important for the document).



$$TFIDF = TF \cdot IDF$$

### How does it work?

Consider two documents (texts):

* `Bonaparte prefered pineapples`

* `Josephine prefered mandarines`

— 5 unique words.

This gives us a 5 (words) x 2 (documents) matrix:

||Bonaparte|prefered|pineapples|Josephine|mandarines|
|:--:|:--:|:--:|:--:|:--:|:--:|
|document 1||||||
|document 2||||||

Computing TF:

||Bonaparte|prefered|pineapples|Josephine|mandarines|
|:--:|:--:|:--:|:--:|:--:|:--:|
|document 1|1/3|1/3|1/3|0|0|
|document 2|0|1/3|0|1/3|1/3|

Computing IDF:

||Bonaparte|prefered|pineapples|Josephine|mandarines|
|:--:|:--:|:--:|:--:|:--:|:--:|
|document 1|$log(2/1)$|$log(2/2)$|$log(2/1)$|||
|document 2||$log(2/2)$||$log(2/1)$|$log(2/1)$|

TFIDF:

||Bonaparte|prefered|pineapples|Josephine|mandarines|
|:--:|:--:|:--:|:--:|:--:|:--:|
|документ 1|0.231|0|0.231|0|0|
|документ 2|0|0|0|0.231|0.231|

Document 1 can therefore be characterized by a vector `[0.231, 0, 0.231, 0, 0]`, document 2 — by a vector `[0, 0, 0, 0.231, 0.231]`.

### What is important to remember while computing TF-IDF?

**Preprocessing**

Get rid of the punctuation, lower and lemmatize your text.

**Stopwords**

Get rid of the stopwords.

### Computing TF-IDF in python

…use the sklearn package.

In [1]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
stops = stopwords.words("russian")

tfidf = TfidfVectorizer(
    analyzer="word", # analyze words or symbols (char)
    stop_words=stops # to pass the list of Russian stopwords from NLTK
)

## An example

Загрузим данные из некоторой заполярной газеты:

In [3]:
import os

In [4]:
path_to_articles = "/content/sample_data/polar_circle"
articles = [os.path.join(path_to_articles, item)
            for item in os.listdir(path_to_articles)
            if item.endswith(".txt")]

In [5]:
articles_texts = []

for article_path in articles:
    with open(article_path, "r", encoding="utf-8") as a_src:
        # the first 6 lines contain metadata, the last line contains the number of comments
        articles_texts.append("\n".join(a_src.readlines()[6:-1]))

Lemmatizing:

!pip install pymorphy2

In [8]:
from nltk.tokenize import wordpunct_tokenize
from pymorphy2 import MorphAnalyzer

In [9]:
morph = MorphAnalyzer()

In [10]:
articles_preprocessed = []
for a_text in articles_texts:
    a_tokens = wordpunct_tokenize(a_text)
    a_lemmatized = " ".join([morph.parse(item)[0].normal_form for item in a_tokens])
    articles_preprocessed.append(a_lemmatized)

Computing TF-IDF:

In [11]:
articles_tfidf = tfidf.fit_transform(articles_preprocessed)
print(f"Matrix of {articles_tfidf.shape[0]} documents and {articles_tfidf.shape[1]} terms")

Matrix of 2 documents and 198 terms


In [14]:
# to look at the terms in the matrix
names = tfidf.get_feature_names_out()
names[10:20]

array(['аварийный', 'август', 'автономный', 'администрация', 'апрель',
       'введение', 'ведомство', 'весь', 'включая', 'возможность'],
      dtype=object)

**keyword extraction**:


In [15]:
import numpy as np

In [16]:
def get_top_tf_idf_words(tfidf_vector, feature_names, top_n):
    sorted_nzs = np.argsort(tfidf_vector.data)[:-(top_n+1):-1]
    return feature_names[tfidf_vector.indices[sorted_nzs]]

In [19]:
feature_names = np.array(tfidf.get_feature_names_out())

for i, article in enumerate(articles_texts):
    # to print the first article
    if i < 1:
        article_vector = articles_tfidf[i, :]
        words = get_top_tf_idf_words(article_vector, feature_names, 10)
        print(article)
        print(words)

Опыт Ямала представлен на конференции «Связь на Русском Севере»

В Сочи завершилась IV конференция «Связь на Русском Севере». Ямал представлял директор департамента информационных технологий и связи региона Олег Ефремов. Он выступил с докладом по устранению «цифрового неравенства» на территории округа. Глава ведомства сообщил, что между правительством автономного округа, Минкомсвязи и ПАО «Ростелеком» заключено трёхстороннее соглашение. Одна из его главных целей – обеспечение равных возможностей для всех жителей региона в использовании современных услуг связи, включая высокоскоростной доступ к сети Интернет. В рамках соглашения предусмотрено строительство точек доступа универсальных услуг связи в населённых пунктах с численностью жителей от 250 до 500 человек. Запланирована модернизация существующих сетей связи в населённых пунктах с численностью населения от 500 до 10 тыс. жителей. Обсуждается проект введения оптоволоконного интернета в деревянные дома. Трёхстороннее соглашение охваты